In [1]:
import torch
import torch.nn as nn
import math
import numpy as np
import copy
from torch.autograd import Variable
import torch.nn.functional as F
import time
from attention import clones, future_mask, MultiHeadedAttention, FeedForwardNetwork, Embeddings, PositionalEncoding, Generator, LayerNorm, SublayerConnection, Encoder, Decoder, EncoderDecoder, make_model, Batch

In [2]:
vocab_size = 10
batch_size = 16
nbatches = 20
d_model = 512

In [3]:
class zrange:
    def __init__(self,batch_size, n_batches, vocab_size, seq_len):
        self.data = torch.from_numpy(np.random.randint(1, vocab_size, size=(batch_size*nbatches, seq_len)))
        self.batch_size = batch_size
        self.n_batches = n_batches
        self.seq_len = seq_len

    def __iter__(self):
        return zrange_iter(self.n_batches, self.batch_size, self.data, self.seq_len)

class zrange_iter:
    def __init__(self, n_batches, batch_size, data, seq_len):
        self.i = 0
        self.n_batches = n_batches
        self.batch_size = batch_size
        self.data = data
        self.seq_len = seq_len

    def __iter__(self):
        # Iterators are iterables too.
        # Adding this functions to make them so.
        return self

    def __next__(self):
        if self.i < self.n_batches:
            idx = self.i*self.batch_size
            src = Variable(self.data[idx:idx+batch_size,:], requires_grad=False)
            tgt = Variable(self.data[idx:idx+batch_size,:], requires_grad=False)
            self.i += 1
            return Batch(src, tgt, 0)
        else:
            raise StopIteration()

In [4]:
z = zrange(batch_size, nbatches, vocab_size, 10)

In [5]:
list(z)

In [6]:
data = torch.from_numpy(np.random.randint(1, vocab_size, size=(1,batch_size*nbatches, d_model)))
data[:, 0] = 1

In [7]:
## testing make_model
model = make_model(vocab_size, vocab_size)


In [8]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [9]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        #print("out shape: ", out.size())
        #print("tgt_y shape: ", batch.trg_y.size())
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens.item()
        tokens += batch.ntokens.item()
        if i % 1 == 0:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    #print("total_loss: ", total_loss)
    #print("total_tokens: ", total_tokens)
    #return total_loss / total_tokens

In [10]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        # x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.item() * norm


In [11]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1).long(), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [12]:
vocab_size = 10
criterion = LabelSmoothing(size=vocab_size, padding_idx=0, smoothing=0.0)
#criterion = nn.CrossEntropyLoss()
model = make_model(vocab_size, vocab_size, N=2)
#data_generator = data_gen(data,vocab_size,batch_size,nbatches,d_model)
data_generator = zrange(batch_size, nbatches, vocab_size,20)
model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))


for epoch in range(0):
    print("epoch: ", epoch)
    model.train()
    run_epoch(data_generator, model, 
              SimpleLossCompute(model.generator, criterion, model_opt))
    model.eval()
    print(run_epoch(data_generator, model, 
                    SimpleLossCompute(model.generator, criterion, None)))


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch:  0
Epoch Step: 0 Loss: 3.279099 Tokens per Sec: 521.182549
Epoch Step: 1 Loss: 3.231597 Tokens per Sec: 850.518499
Epoch Step: 2 Loss: 2.961998 Tokens per Sec: 798.629823
Epoch Step: 3 Loss: 2.682320 Tokens per Sec: 921.745089
Epoch Step: 4 Loss: 2.476744 Tokens per Sec: 916.214631
Epoch Step: 5 Loss: 2.278021 Tokens per Sec: 828.295133
Epoch Step: 6 Loss: 2.152127 Tokens per Sec: 931.209862
Epoch Step: 7 Loss: 1.794962 Tokens per Sec: 849.186234
Epoch Step: 8 Loss: 1.488375 Tokens per Sec: 866.637678
Epoch Step: 9 Loss: 1.251848 Tokens per Sec: 938.499847
Epoch Step: 10 Loss: 1.059791 Tokens per Sec: 929.086059
Epoch Step: 11 Loss: 0.997878 Tokens per Sec: 909.817594
Epoch Step: 12 Loss: 0.796727 Tokens per Sec: 864.163742
Epoch Step: 13 Loss: 0.735336 Tokens per Sec: 876.911851
Epoch Step: 14 Loss: 0.634672 Tokens per Sec: 840.749721
Epoch Step: 15 Loss: 0.580447 Tokens per Sec: 845.326995
Epoch Step: 16 Loss: 0.435455 Tokens per Sec: 918.951008
Epoch Step: 17 Loss: 0.372148 T

KeyboardInterrupt: 

In [13]:
def greedy_decode(model, source, source_mask, max_decode_len, start_symbol):
    encoder_outputs = model.encode(source, source_mask) # todo(annhe): check source_masking
    print("encoder outputs shape: ", encoder_outputs.size())
    ys = torch.ones(1,1).fill_(start_symbol).type_as(source.data) # create a 1x1 array with element '<sos>'
    for i in range(max_decode_len-1):
        print("i: ", i)
        #print("source mask shape: ", source_mask.size())
        print("ys shape: ", ys.size())
        #print("source shape: ", source.size())
        output_mask = torch.Tensor(future_mask(ys.size(1)).type_as(src.data))
        #print("output mask shape: ", output_mask.size())
        output = model.decode(encoder_outputs, source_mask, torch.Tensor(ys),
                              output_mask)
        print(output.size())
        # output is size N x L x D
        # we need to pass it through the generator
        prob = model.generator(output[:,-1,:])
        print(prob.size())
        _, vocab_symbol = torch.max(prob, dim=1)
        vocab_symbol = vocab_symbol.detach().unsqueeze(0)
        ys = torch.cat([ys, vocab_symbol], dim=1)
model.eval()
src = Variable(torch.LongTensor([[1,2,3,4,5,6,7]]) )
src_mask = Variable(torch.ones(1, 1, 7) )
greedy_decode(model, src, src_mask, max_decode_len=7, start_symbol=1)
    

encoder outputs shape:  torch.Size([1, 7, 512])
i:  0
ys shape:  torch.Size([1, 1])
torch.Size([1, 1, 512])
torch.Size([1, 10])
i:  1
ys shape:  torch.Size([1, 2])
torch.Size([1, 2, 512])
torch.Size([1, 10])
i:  2
ys shape:  torch.Size([1, 3])
torch.Size([1, 3, 512])
torch.Size([1, 10])
i:  3
ys shape:  torch.Size([1, 4])
torch.Size([1, 4, 512])
torch.Size([1, 10])
i:  4
ys shape:  torch.Size([1, 5])
torch.Size([1, 5, 512])
torch.Size([1, 10])
i:  5
ys shape:  torch.Size([1, 6])
torch.Size([1, 6, 512])
torch.Size([1, 10])
